 Исследуйте взаимосвясь между продажами
 категорий 'Чипсы, снеки, попкорн' и напитками:'Вода', 'Газированные напитки',  '
Соки, нектары', 'Лимонады', 'Безалкогольное пиво', 'Вино', 'Пиво'. С чем сильнее коррелируют
 'Чипсы, снеки, попкорн'?

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

categories = pd.read_pickle('data/categories.pkl')
orders = pd.read_pickle('data/orders.pkl')
category = categories[['Category', 'ItemID', 'ParentCategory']]

df_exploded = orders.explode('items')

In [7]:

new_df = pd.json_normalize(df_exploded['items'])
new_df['OrderID'] = df_exploded['OrderID'].values
# print(new_df)
# print(category)

filter_list = ['чипсы, снеки, попкорн', 'вода', 'газированные напитки',
               'соки, нектары', 'лимонады', 'безалкогольное пиво', 'вино', 'пиво']
# filter_list = ['чипсы, снеки, попкорн']

# list_of_snacks = ['чипсы', 'снеки', 'попкорн', 'чипсы, снеки, попкорн']
list_of_snacks = ['чипсы, снеки, попкорн,']

list_of_drinks = ['вода', 'газированные напитки', 'соки, нектары', 'лимонады', 'безалкогольное пиво', 'вино', 'пиво']


final_df = pd.merge(new_df, category, on='ItemID')
final_df['Category'] = final_df['Category'].str.lower()

final_df = final_df[(final_df['Category'].isin(filter_list))]
# print(final_df)


# Объединение по OrderID
result = final_df.groupby('OrderID').agg({
    'ItemID': 'count',  # количество товаров в заказе
    'price': 'sum',     # общая цена
    'quantity': 'sum',  # общее количество
    'Category': lambda x: ', '.join(x.unique()),  # все уникальные категории
    'ParentCategory': 'first'  # первая родительская категория
}).reset_index()



# Фильтрация DataFrame
# Проверяем, содержит ли Category закуски и другие значения
def contains_snacks_and_more(category):
    # Проверяем, есть ли закуски в категории
    has_snacks = any(snack in category for snack in list_of_snacks)
    # Проверяем, есть ли другие значения (не только закуски)
    has_other_items = len(category.split(',')) > len(list_of_snacks)
    return has_snacks and has_other_items

# Применяем фильтрацию
filtered_df = result[result['Category'].apply(contains_snacks_and_more)]

# Выводим отфильтрованный DataFrame
print(filtered_df)

        OrderID  ItemID   price  quantity                                           Category ParentCategory
72     89730791       5  256.12        32  чипсы, снеки, попкорн, вода, газированные напитки        Бакалея
129    90113514       6  632.84        16  соки, нектары, газированные напитки, чипсы, сн...        Напитки
344    90560203       2  189.80         2                        чипсы, снеки, попкорн, вода        Бакалея
423    90612856       3  279.70         3  чипсы, снеки, попкорн, соки, нектары, газирова...        Бакалея
491    90653028      10  504.47        19  соки, нектары, лимонады, вода, чипсы, снеки, п...        Напитки
...         ...     ...     ...       ...                                                ...            ...
10324  92585988       3  237.00         3               чипсы, снеки, попкорн, соки, нектары        Бакалея
10475  92618349       2  209.83         7        чипсы, снеки, попкорн, газированные напитки        Бакалея
10682  92654277       2  110

Внимание не доделал, не понимаю как все соеденить 